Minor script to prepare catalogs for the voidiness analysis

In [19]:
import pandas as pd
from astropy.cosmology import WMAP9 as cosmo # Used to calculate comoving dist
from footprint_filter import filter_by_footprint
from astropy.table import Table
import numpy as np


In [20]:
voids = pd.read_excel('exported_dataFrames/voids.xlsx')

In [21]:
def filter_by_redshift(voids_df, data_df):
    too_close = data_df.z < min(voids_df.z)
    data_df = data_df[~too_close]

    too_far = data_df.z > max(voids_df.z)
    return data_df[~too_far]

def add_cmvd(data_df):
    """Adds the comoving distance to data_df"""
    cmvd = cosmo.comoving_distance(data_df['z']) # Comoving distance to void center
    data_df['cmvd_Mpc'] = cmvd.value # add it to data table    
    return data_df

def filter_by_z_ra_dec(data_df, voids, cmvd_add = True, footprint_points_fn = "exported_dataFrames/footprint_points.xlsx"):
    
    if type(data_df) == str:
        data_df = pd.read_excel(data_df)
    else:
        assert isinstance(data_df, pd.DataFrame)
        
    data_df = filter_by_redshift(voids,data_df)

    if cmvd_add:
        data_df = add_cmvd(data_df)
    data_df = filter_by_footprint(data_df, footprint_points_fn)
    return data_df

In [25]:
## 4LAC catalog
four_lac_GRS = filter_by_z_ra_dec('More GRS/FINALCorrectedRedshifts.xlsx', voids)

# Save to excel file
four_lac_GRS.to_excel('exported_dataFrames/z_ra_dec_filtered_4lac.xlsx', index=False)

In [26]:
## xray catalog
# updating to Swift BAT 105 Month Catalog
# source: https://swift.gsfc.nasa.gov/results/bs105mon/
# Only the RAdeg, DEdec and z columns were properly cleaned for whitespace since 
# its all we need
xray_agns = pd.read_excel('agn data/SBAT105mo.xlsx')

# Converting white space to 0
for i, _ in xray_agns.iterrows():
    if type(xray_agns.z[i]) == str:
        xray_agns.at[i, 'z'] = 0
# Typecast to single type
xray_agns.astype({'z':'float64'}).dtypes
# Save it
xray_agns.to_excel('exported_dataFrames/xray_agns.xlsx', index=False)

#Filter and Save
xray_agns = filter_by_z_ra_dec('exported_dataFrames/xray_agns.xlsx', voids)

xray_agns.to_excel('exported_dataFrames/z_ra_dec_filtered_xray_agns.xlsx', index = False)

In [27]:
## Massive SDSS DR16 Catalog

dat = Table.read('SDSS Data Used/DR16Q_Superset_v3.fits', format='fits')
# We lose data on this steps since some colums have multiple data points per row
multidim_cols = [name for name in dat.colnames if len(dat[name].shape) >1]# For bookkeeping purposes

# Grab the non multidimensional columsn
singledim_cols = [name for name in dat.colnames if len(dat[name].shape) <=1] 
sdss_dr16 = dat[singledim_cols].to_pandas()

In [28]:
# NOTE: Check coordinate system used. Ensure all data tables are using the same systme
# Note. Some redshifts are negative. Why? They are specific negative values. 
sdss_dr16 = sdss_dr16.rename(columns={"RA": 'RAdeg', "DEC": "DEdeg", "Z":'z'})


# Convert bytes to string
sdss_dr16['AUTOCLASS_PQN'] = sdss_dr16['AUTOCLASS_PQN'].apply(lambda x: x.decode('utf-8'))

# Filter by object type. We only want to keep  QSO's and Galaxies
is_quasar = sdss_dr16.AUTOCLASS_PQN == "QSO"
is_galaxy = sdss_dr16.AUTOCLASS_PQN == "GALAXY"

# Filter by galaxy or quasar
sdss_dr16 = sdss_dr16[is_galaxy | is_quasar]

In [29]:
# Now the big fanxy filter

sdss_dr16 = filter_by_z_ra_dec(sdss_dr16, voids)

In [30]:
sdss_dr16.to_excel('exported_dataFrames/z_ra_dec_filtered_SDSS_DR16.xlsx', index=False)

In [31]:
# ROMABZCAT catalog
# https://heasarc.gsfc.nasa.gov/W3Browse/galaxy-catalog/romabzcat.html#source_number
# Soft xrays I believe
soft_xrays = pd.read_excel('agn data/ROMABZCAT.xlsx')

for i, _ in soft_xrays.iterrows():
    if np.isnan(soft_xrays.z[i]):
        soft_xrays.at[i, 'z'] = 0

# save it
filter_by_z_ra_dec(soft_xrays, voids).to_excel('exported_dataFrames/z_ra_dec_filtered_ROMABZCAT.xlsx', index=False)
